In [1]:
# =================== Previous Code(with small change) from Dataset Provider bellow =======================

In [2]:
from open3d import *
import numpy as np
from scipy.io import loadmat

import torch
import torch.utils.data as data

import os
from os import listdir
from os.path import isfile, join

In [3]:
def load_directory_names(filename):
    
    directory_list = []
    
    file = open(filename, 'r') 
    lines = file.readlines() 

    for line in lines: 
        line_string = line.strip()
        split_string = line_string.split(" ", 2)
        
        directory_list.append(split_string[1])
        
    return directory_list


# get all filenames in a given folder 

def get_all_filenames_from_directory(directory_path):
    onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
    
    onlyfiles.sort()
    
    return onlyfiles


def seperate_file_list_into_pcd_and_label(file_list):
    
    file_list.sort()
    
    pcd_list = [];
    label_list = []
    
    suffix_pcd = ".pcd"
    suffix_mat = ".mat"
    
    for file in file_list:
        
        if(file.endswith(suffix_pcd)):
            pcd_list.append(file)
            
        if(file.endswith(suffix_mat)):
            label_list.append(file)
            
    
    pcd_list.sort()
    label_list.sort()
    
    return pcd_list, label_list


In [4]:
class Dataset(data.Dataset):
    
    def __init__(self, dataset_root, dataset_folders, number_of_points, categories):
        
        self.dataset_root = dataset_root
        
        # list of folders in root folder
        self.dataset_folders = load_directory_names(dataset_folders)
        self.categories = categories
        self.number_of_points = number_of_points
        
        self.pointcloud_list = []
        self.label_list = []
        
        # load labels and pointclouds into self.pointcloud_list and self.label_list
        self.load_point_clouds()
        
        
    def load_point_clouds(self):
        
        # iterate over all folders
        for folder in self.dataset_folders:
            
            for j in range(0,len(self.categories)):
                # if first part of folder name equals category name
                if(folder.split('_')[0] == self.categories[j]):
                    # get all filenames in given folder
                    file_list = get_all_filenames_from_directory(self.dataset_root + '/' + folder)
                    
                    pcd_files, label_files = seperate_file_list_into_pcd_and_label(file_list)
                    
                    # iterate over all files in folder, already ensured that pcd_files and label_files have same length
                    for i in range (0, int(len(pcd_files))):
                        
                        # load pointcloud
                        pcd = open3d.io.read_point_cloud(self.dataset_root + '/' + folder + '/' + pcd_files[i])
                        # load labels
                        labels = loadmat(self.dataset_root + '/' + folder + '/' + label_files[i])
                        
                        self.pointcloud_list.append(pcd)
                        self.label_list.append(labels)
                        
    def __getitem__(self, index):

        labels = self.label_list[index]['data'][0]
        points = np.array(self.pointcloud_list[index].points)
        
        # randomly choose number of points form point cloud and labels 
        choice = np.random.choice(len(points), self.number_of_points, replace=True)
        
        points = points[choice, :]
        labels = labels[choice]
        
        points = points.astype(np.float32)

        torch_labels = torch.from_numpy(labels)
        torch_points = torch.from_numpy(points)
        choice=0
        return torch_points, torch_labels 
    
    def __len__(self):
        return len(self.pointcloud_list)

In [5]:
# =================== Previous Code(with small change) from Dataset Provider above =======================

In [6]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch as torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from pointnet import PointNetDenseCls
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [7]:
from sklearn.metrics import f1_score, precision_score, accuracy_score

def evaluate_f_score(seg, pred):
    
    f1 = f1_score(seg, pred_choice[0], average='weighted')
    precision = precision_score(seg, pred_choice[0], average='weighted')
    accuracy = accuracy_score(seg, pred_choice[0])
    
    return f1, precision, accuracy

In [9]:
dict_cat = {
    
    0 : ['bowl'],
    1 : ['cup'],
    2 : ['hammer'],
    3 : ['knife'],
    4 : ['ladle'],
    5 : ['mallet'],
    6 : ['mug'],
    7 : ['saw'],
    8 : ['scissors'],
    9 : ['scoop'],
    10 : ['shears'],
    11 : ['shovel'],
    12 : ['spoon'],
    13 : ['tenderizer'],
    14 : ['trowel'],
    15 : ['turner'],   
    
}

In [10]:
final_list = []

for i in range (0, len(dict_cat)):     
    
    d = Dataset('./dataset', './tool_categories_test.txt', 2500, dict_cat[i])
    
    result_f = []
    result_precision = []
    result_accuracy = []

    for j in range(0 , len(d)):

        # get the points and segmentation
        point, seg = d[j]

        point_np = point.numpy()

        """ k is number of categories in our case 7
          1 - 'grasp'
          2 - 'cut'
          3 - 'scoop'
          4 - 'contain'
          5 - 'pound'
          6 - 'support'
          7 - 'wrap-grasp'
        """
        # load segmentation model
        classifier = PointNetDenseCls(k = 7)
        classifier.load_state_dict(torch.load('seg_total/seg_model_34.pth', map_location=torch.device('cpu')))
        classifier.eval()

        point = point.transpose(1,0).contiguous()

        # predict
        point = Variable(point.view(1, point.size()[0], point.size()[1]))
        pred, _ = classifier(point)
        pred_choice = pred.data.max(2)[1]

        # decrease all seg values -1 because seg values are from 1-7 and predicted values are from 0-6
        seg = seg - 1
        
        f1 = 0.0
        precision = 0.0
        accuracy = 0.0

        f1, precision, accuracy = evaluate_f_score(seg, pred_choice)
        result_f.append(f1)
        result_precision.append(precision)
        result_accuracy.append(accuracy)
    
    
    print("F-score for " + dict_cat[i][0] + ": " + str(sum(result_f)/len(result_f)))
    print("Precision for " + dict_cat[i][0] + ": " + str(sum(result_precision)/len(result_precision)))
    print("Accuracy for " + dict_cat[i][0] + ": " + str(sum(result_accuracy)/len(result_accuracy)))
    final_list.append(sum(result_f)/len(result_f))
    final_list.append(sum(result_precision)/len(result_precision))
    final_list.append(sum(result_accuracy)/len(result_accuracy))

/Users/manuel/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/manuel/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/manuel/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F-score for bowl: 0.7165847615606997
Precision for bowl: 0.7213114754098361
Accuracy for bowl: 0.7151159112825457
F-score for cup: 0.9715515215824503
Precision for cup: 0.9733141964175855
Accuracy for cup: 0.970787476979741
F-score for hammer: 0.8935661979380589
Precision for hammer: 0.9025074491459129
Accuracy for hammer: 0.8949225688073394
F-score for knife: 0.9469881885625803
Precision for knife: 0.954534201917379
Accuracy for knife: 0.9475311657414747
F-score for ladle: 0.5377185811473147
Precision for ladle: 0.5654785957317834
Accuracy for ladle: 0.5195141762452109
F-score for mallet: 0.9183612884407893
Precision for mallet: 0.9289441743799715
Accuracy for mallet: 0.9157821782178212
F-score for mug: 0.848262951678244
Precision for mug: 0.8836224583784187
Accuracy for mug: 0.8450979610330778
F-score for saw: 0.16134809470019112
Precision for saw: 0.1933479128503106
Accuracy for saw: 0.16933831775700942
F-score for scissors: 0.8914105218237915
Precision for scissors: 0.9179839229802